<a href="https://colab.research.google.com/github/Gabriel07201/Alura/blob/main/Data%20Science/Word%20Embedding/word2vec_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!python -m spacy download pt_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-10 00:01:58.637333: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 13.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [16]:
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Cursos Alura/Word2Vec/modelo_cbow.txt')
w2v_modelo_sg = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Cursos Alura/Word2Vec/modelo_skipgram.txt')

In [17]:
import spacy

nlp = spacy.load('pt_core_news_sm', disable=['paser', 'ner', 'tagger', 'textcat'])
def tokenizador(texto):
  doc = nlp(texto)
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())
  return tokens_validos

texto = 'Rio de Janeiro 1242341 ****** @#$  é uma cidade ]maravilhosa!'
tokens = tokenizador(texto)
tokens

['rio', 'janeiro', 'cidade', 'maravilhosa']

In [15]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras, modelo):
  vetor_resultante = np.zeros(300)
  for pn in palavras:
    try:
      vetor_resultante += modelo.get_vector(pn)
    except KeyError:
      pass
  return vetor_resultante

In [19]:
vetor_texto =  combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(len(vetor_texto))
print(vetor_texto)

300
[-1.17634246 -1.14019378 -0.9090994  -2.36575409  1.15502058  0.14122358
  0.52115718 -1.06832685  0.67814177 -2.49323627 -0.3578898  -2.35580641
  0.09010491 -2.24757112 -1.43412417 -0.19769326  0.2089836   0.02600077
 -0.00425345  0.71724978  0.9109519   0.05877449  1.40488479 -0.86572702
  0.06373351 -0.33606984  0.3751256  -0.78106537  0.43073519  0.69871645
 -0.72812381  1.6278627   2.35009569 -0.25385038 -2.20157483 -0.43948415
 -1.61391167  0.55847844 -0.25029335 -0.63947171  0.30373776 -1.20318091
  0.12003119 -0.61013848  0.40871848 -0.75491042 -1.76372492 -0.56270982
 -1.65017164  2.75028192 -1.34849083  1.07781331 -0.43790426 -0.9872654
 -0.97565861 -2.60821539 -1.52834965  1.11400255 -0.13607264  0.39844404
  2.1084576   0.06216897  2.14688119 -0.25600845 -0.46715367  0.97735345
 -4.13654983  1.03018686 -1.90932251 -0.16238049  0.6851853   0.30515751
 -1.54601949  2.11195937 -2.3290835  -1.74233409  0.99058561 -1.10934395
 -0.52660003  0.6439026  -0.54667366 -0.75460219

In [20]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x, y))
  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)
  return matriz

In [21]:
import pandas as pd

artigo_treino = pd.read_csv(r'/content/drive/MyDrive/Cursos Alura/Word2Vec/treino.csv')
artigo_teste = pd.read_csv(r'/content/drive/MyDrive/Cursos Alura/Word2Vec/teste.csv')

In [22]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino['title'], w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste['title'], w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  rl = LogisticRegression(max_iter=800)
  rl.fit(x_treino, y_treino)
  categorias = rl.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return rl

rl_cbow = classificador(w2v_modelo_cbow, matriz_vetores_treino_cbow, artigo_treino['category'], matriz_vetores_teste_cbow, artigo_teste['category'])

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.83      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.81      0.69     20513
weighted avg       0.82      0.78      0.80     20513



In [24]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino['title'], w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste['title'], w2v_modelo_sg)
rl_sg = classificador(w2v_modelo_sg, matriz_vetores_treino_sg, artigo_treino['category'], matriz_vetores_teste_sg, artigo_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.65      0.80      0.72      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.89      0.25       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [27]:
import pickle

with open('/content/drive/MyDrive/Cursos Alura/Word2Vec/rl_cbow.pkl', 'wb') as f:
  pickle.dump(rl_cbow, f)

In [28]:
import pickle

with open('/content/drive/MyDrive/Cursos Alura/Word2Vec/rl_sg.pkl', 'wb') as f:
  pickle.dump(rl_sg, f)